# <font color='red'> Description du projet </font>

## <font color='blue'>Présentation du problème </font>

L’objectif de ce projet est d’estimer **les temps de réponse et de mobilisation** de la Brigade des Pompiers de Londres. La brigade des pompiers de Londres est le service d'incendie et de sauvetage le plus actif du Royaume-Uni  et l'une des plus grandes organisations de lutte contre l'incendie et de sauvetage au monde.

Le premier jeu de données fourni contient les détails de chaque incident traité depuis janvier 2009. Des informations sont fournies sur la date et le lieu de l'incident ainsi que sur le type d'incident traité. Il est composé de deux fichiers

*   LFB Incident data from 2009 - 2017.xlsx
*   LFB Incident data from 2018 onwards.csv

Le second fichier peut-être récupéré à l'aide du lien : 'https://data.london.gov.uk/download/london-fire-brigade-incident-records/f5066d66-c7a3-415f-9629-026fbda61822/LFB%20Incident%20data%20from%202018%20onwards.csv.xlsx' pour avoir la dernière version du fichier. En effet, les données sont mises à jour tous les mois. Il faut compter au moins 7 minutes pour la lecture des données.

<br>

Le second jeu de données contient les détails de chaque camion de pompiers envoyé sur les lieux d'un incident depuis janvier 2009. Des informations sont fournies sur l'appareil mobilisé, son lieu de déploiement et les heures d'arrivée sur les lieux de l'incident. Il est composé de trois fichiers

*   LFB Mobilisation data from January 2009 - 2014.xlsx
*   LFB Mobilisation data from 2015 - 2020.xlsx
*   LFB Mobilisation data from January 2009 - 2014.xlsx

Le dernier fichier peut-être récupéré à l'aide du lien : 'https://data.london.gov.uk/download/london-fire-brigade-mobilisation-records/3ff29fb5-3935-41b2-89f1-38571059237e/LFB%20Mobilisation%20data%202021%20-%202024.xlsx' pour avoir la dernière version du fichier (mise à jour mensuelle). Il faut compter environ 17 minutes pour la lecture des données.

## <font color='blue'> Etapes précédentes </font>

*   1 - Exploration des données : premières analyses, concaténation des différents fichiers puis jointure des 2 types de données (incident / mobilisation)
*   2 - Data visualisation.ipynb : visualisation des données, étude de la variable à prédire (temps de réponse total) en fonction des variables explicatives, création d'un jeu de données pour la modélisation
*   5 - Preprocessing final : création des jeux de données pour la modélisation faite dans ce notebook
*   6 - Modelisation Preproc final - Pred Continue : modélisation de la variable continue `TotalResponseTime_BC`
*  7 - Modelisation Preproc final - Pred 10Cat : modélisation de la variable continue `ResponseTimeCategory`
*  8 - Modelisation Preproc final - Pred 6Cat : modélisation de la variable continue `ResponseTimeCategory2`

*Nota Bene* : Les notebooks numérotés 3 et 4 ont été conservés pour montrer un premier travail de modélisation qui a été abandonné par la suite (voir détails ci-dessous)


## <font color='blue'>Etapes dans ce notebook </font>

Dans ce notebook, nous avons modélisé le temps de réponse, après sa binarisation selon la limite de 6 minutes.

Pour évaluer nos modèles (et les comparer) nous utilisons l'exactitude, la précision pour chaque classe ainsi que le rappel et le f1-score. Ces mesures sont calculées sur le jeu de données de validation.

# <font color='red'>1) Préparation de l'environement de travail </font>

## <font color='blue'>Installation des modules </font>

In [ ]:
#!pip install matplotlib
#!pip install Seaborn
#!pip install openpyxl
#!pip install scipy
#!pip install geopandas
#!pip install scikit-learn
#!pip install statsmodels
#!pip install folium
#!pip install plotly
#!pip install --upgrade seaborn
#!pip install jupyter
#!pip install nbformat
!pip install lightgbm
!pip install xgboost
#!pip optuna

## <font color='blue'>Importation des bibliothèques </font>

In [ ]:
import pandas as pd  #Pour les dataframe
import numpy as np #Pour le calcul numérique
import datetime as dt # Pour le calcul sur les dates

## Libraries pour les graphiques
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D # graphique 3D
import plotly.express as px  #graphique 3D dynamique
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as offline

## Libraries pour statistiques
# régression linéaire
import statsmodels.api as sm
from statsmodels.formula.api import ols
# tests statistiques
from scipy.stats import shapiro , kstest # tests de sur la normalité de la distributin
from scipy.stats import bartlett # tests sur les variances
from scipy.stats import kruskal #  comparaison des médianes
from scipy.stats import spearmanr
from scipy.stats import loguniform, uniform
from scipy import stats # notamment pour boxplot
# metrics
from sklearn.metrics import f1_score, accuracy_score, make_scorer, classification_report, recall_score, precision_recall_fscore_support

# Libraries divers
from copy import deepcopy  # gestion des copies

# Pour la séparation du jeu de données
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, cross_validate, RandomizedSearchCV, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, LogisticRegression

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
# import optuna


## <font color='blue'>Liaison avec le drive (pour travailler sur GoogleColab) </font>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# <font color='red'>2) Données pour la modélisation </font>

## <font color='blue'>2.a) Chargement </font>


Chargement des données depuis GoogleColab

In [ ]:
train = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset.csv', low_memory=False)
validation = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset.csv', low_memory=False)

train_reduit = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Train_Dataset.csv', low_memory=False)
validation_reduit = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Validation_Dataset.csv', low_memory=False)

train_reduit2 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Train_Dataset.csv', low_memory=False)
validation_reduit2 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Validation_Dataset.csv', low_memory=False)


Chargement des données depuis un emplacement en local

In [ ]:
train = pd.read_csv('../Data/Datapreprocessing/Complete/Train_Dataset.csv', low_memory=False)
validation = pd.read_csv('../Data/Datapreprocessing/Complete/Validation_Dataset.csv', low_memory=False)

train_reduit = pd.read_csv('../Data/Datapreprocessing/Reduit2/Train_Dataset.csv', low_memory=False)
validation_reduit = pd.read_csv('../Data/Datapreprocessing/Reduit2/Validation_Dataset.csv', low_memory=False)

train_reduit2 = pd.read_csv('../Data/Datapreprocessing/Reduit1/Train_Dataset.csv', low_memory=False)
validation_reduit2 = pd.read_csv('../Data/Datapreprocessing/Reduit1/Validation_Dataset.csv', low_memory=False)

## <font color='blue'>2.b) Sélection des variables </font>

In [ ]:
# X : variable explicative du modele --> on supprime IncidentNumber et les 4 variables à prédire
# y : la variable à prédire est ResponseTimeCategory
X_train = train.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_train = train[['ResponseTimeBinary']]

X_val = validation.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_val = validation[['ResponseTimeBinary']]


# même principe sur le jeux de données réduit (pour la recherche d'hyperparamètres)
X_train_r = train_reduit.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_train_r = train_reduit[['ResponseTimeBinary']]

X_val_r = validation_reduit.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_val_r = validation_reduit[['ResponseTimeBinary']]

# même principe sur le jeux de données réduit (pour la recherche d'hyperparamètres)
X_train_r2 = train_reduit2.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_train_r2 = train_reduit2[['ResponseTimeBinary']]

X_val_r2 = validation_reduit2.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_val_r2 = validation_reduit2[['ResponseTimeBinary']]



# <font color='red'> 3) Méthodes
 </font>







Dans la suite de ce notebook, nous utilisons les méthodes définies ici

In [ ]:
### evaluation du modèle
# cette méthode calcul les différentes métriques permettant l'évaluation du modèle
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n{model_name}")
    precision, recall, fscore, _ = precision_recall_fscore_support(y_true, y_pred)
    print("accuracy :", np.round(accuracy_score(y_true, y_pred),4))
    print("precision :", np.round(precision,4))
    print("recall :", np.round(recall,4))
    print("f1 score :", np.round(fscore,4))


In [ ]:
### estimation des paramètres, prédiction et calcul des métriques d'évaluation
def train_and_evaluate_model(model, model_name, X_train, y_train, X_val, y_val):
    y_train = y_train.to_numpy().ravel() if isinstance(y_train, pd.DataFrame) else y_train
    y_val = y_val.to_numpy().ravel() if isinstance(y_val, pd.DataFrame) else y_val
    # Entraînement du modèle
    model.fit(X_train, y_train)

    # Prédiction sur l'ensemble de validation
    y_pred_val = model.predict(X_val)

    # Évaluation sur l'ensemble de validation
    evaluate_model(y_val, y_pred_val, model_name)

    return y_pred_val

In [ ]:
### recherche des hyper paramètres en utilisant RandomizedSearchCV
def find_best_params_and_predict(model, param_distributions, X_train, y_train, X_val, y_val, model_name, n_iter):
    """
    Effectue une recherche des meilleurs paramètres pour un modèle donné avec RandomizedSearchCV, et retourne les prédictions.
    """
    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=n_iter,               # Nombre d'itérations de recherche aléatoire
        scoring='accuracy',      # métrique à utiliser pour le choix d'hyper paramètres
        cv=3,                    # Nombre de sub dataset pour la validation croisée
        n_jobs=-1,               # Utilisation de tous les cœurs disponibles
        verbose=1,               # Affiche les logs de progression
        random_state=42          # Pour garantir la reproductibilité
    )

    # Exécute la recherche
    randomized_search.fit(X_train, y_train)

    # Meilleurs paramètres et score
    print(f"\n{model_name} - Best Parameters: {randomized_search.best_params_}")
    print(f"{model_name} - Best Cross-Validation Score: {randomized_search.best_score_:.4f}")

    # Meilleur modèle entraîné
    best_model = randomized_search.best_estimator_

    # Prédictions sur l'ensemble de validation
    y_pred = best_model.predict(X_val)

    # Évaluation
    evaluate_model(y_val, y_pred, model_name)

    return y_pred, randomized_search.best_params_


In [ ]:
### seconde fonction de recherche des hyper paramètres en utilisant RandomizedSearchCV
### 5 datasets de cross validation au lieu de 3 + changement du random_state
def find_best_params_and_predict2(model, param_distributions, X_train, y_train, X_val, y_val, model_name, n_iter):
    """
    Effectue une recherche des meilleurs paramètres pour un modèle donné avec RandomizedSearchCV, et retourne les prédictions.
    """
    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=n_iter,               # Nombre d'itérations de recherche aléatoire
        scoring='accuracy',      # métrique à utiliser pour le choix d'hyper paramètres
        cv=5,                    # Nombre de sub dataset pour la validation croisée
        n_jobs=-1,               # Utilisation de tous les cœurs disponibles
        verbose=1,               # Affiche les logs de progression
        random_state=142          # Pour garantir la reproductibilité
    )

    # Exécute la recherche
    randomized_search.fit(X_train, y_train)

    # Meilleurs paramètres et score
    print(f"\n{model_name} - Best Parameters: {randomized_search.best_params_}")
    print(f"{model_name} - Best Cross-Validation Score: {randomized_search.best_score_:.4f}")

    # Meilleur modèle entraîné
    best_model = randomized_search.best_estimator_

    # Prédictions sur l'ensemble de validation
    y_pred = best_model.predict(X_val)

    # Évaluation
    evaluate_model(y_val, y_pred, model_name)

    return y_pred, randomized_search.best_params_

# <font color='red'> 4) Classification
 </font>


## <font color='blue'>4.a) Régression logistique </font>

Le modèle le plus "simple" applicable à une variable binaire est la régression logistique. L'accuracy est de 76,6%.

In [ ]:
LR_model = LogisticRegression(max_iter=1000, random_state=42)
y_pred_LR=train_and_evaluate_model(LR_model, "Logistic Regression", X_train, y_train, X_val, y_val)


Logistic Regression
accuracy : 0.7657
precision : [0.7857 0.6684]
recall : [0.9202 0.3905]
f1 score : [0.8477 0.4929]


## <font color='blue'>4.b) Support Vector Machine (SVM) </font>

Après plus de 12h, nous avons interrompu l'algorithme SVM. Nous avons tenté plusieurs configurations mais n'avons jamais eu de résultats. Ce type de modèle est donc écarté.

In [ ]:
svm_model = SVC(random_state=42)
y_pred_svm = train_and_evaluate_model(svm_model, "SVM", X_train, y_train, X_val, y_val)

## <font color='blue'>4.c) K-plus proches voisins </font>

### Première estimation

Nous avons fait une première estimation avec l'algorithme des k-plus proches voisins en utilisant les paramètres par défaut. L'accuracy est de 78,9%

In [ ]:
knn_model = KNeighborsClassifier()
y_pred_KNC=train_and_evaluate_model(knn_model, "k-Nearest Neighbors", X_train, y_train, X_val, y_val)


k-Nearest Neighbors
accuracy : 0.7894
precision : [0.8309 0.6634]
recall : [0.8822 0.5639]
f1 score : [0.8558 0.6096]


### Recherche des hyper paramètres

Pour améliorer ces résultats, nous avons fait une recherche d'hyper-paramètres sur un jeu de données réduit contenant un peu moins de 140 000 observations pour l'entraînement (`y_train_r`, `X_train_r`) et 30 000 pour la validation (`yval_r`, `X_val_r`).

In [ ]:
knn_param_distributions = {
    'n_neighbors': [5, 10, 20, 50],
    'weights': ['uniform', 'distance'],
    'leaf_size': [1, 5, 15, 30],
    'p': [1, 2, 5]
}

knn_model_best = KNeighborsClassifier()

y_pred_Bestknn, best_params_knn= find_best_params_and_predict(knn_model_best, knn_param_distributions, X_train_r, y_train_r, X_val_r, y_val_r, "Optimized Random Forest Regressor", 50)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().




Optimized Random Forest Regressor - Best Parameters: {'weights': 'uniform', 'p': 5, 'n_neighbors': 50, 'leaf_size': 30}
Optimized Random Forest Regressor - Best Cross-Validation Score: 0.7949

Optimized Random Forest Regressor
accuracy : 0.7933
precision : [0.8114 0.7256]
recall : [0.9171 0.507 ]
f1 score : [0.861  0.5969]


Sur le jeu de données "complet", l'accuracy est de 80,3%. A noter, l'exécution de ce modèle est particulièrement longue à cause du nombre de voisins (`n_neigbors=50`)

In [ ]:
knn_model = KNeighborsClassifier(weights='uniform', p=5, n_neighbors=50, leaf_size=30)
y_pred_KNC=train_and_evaluate_model(knn_model, "k-Nearest Neighbors", X_train, y_train, X_val, y_val)


k-Nearest Neighbors
accuracy : 0.8028
precision : [0.8251 0.7208]
recall : [0.9157 0.5287]
f1 score : [0.868 0.61 ]


## <font color='blue'>4.d) Arbre de décision </font>

Nous avons uniquement fixé le `random_state`, utilisant les autres paramètres par défaut. L'accuracy est de 75,9%.

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)
y_pred_DT= train_and_evaluate_model(dt_model, "Decision Tree", X_train, y_train, X_val, y_val)


Decision Tree
accuracy : 0.7586
precision : [0.8159 0.5964]
recall : [0.8514 0.5334]
f1 score : [0.8333 0.5631]


La distance est la variable explicative la plus importante du modèle.

In [ ]:
feat_imp = pd.DataFrame({'importance': dt_model.feature_importances_}, index=dt_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.787373
ratioStd,0.030833
Bor_inc_rep,0.015841
H1117,0.013128
PropCat_Dwelling,0.013122
PropCat_Non Residential,0.010072
PropCat_Road Vehicle,0.008471
Stat_resp_rep,0.007388
PropCat_Outdoor Structure,0.007143
PropCat_Outdoor,0.006967


## <font color='blue'>4.e) Forêt aléatoire </font>

### Première estimation

En utilisant les hyper-paramètres par défaut, l'accuracy obtenu pour le modèle de Forêt Aléatoire (76,1%) est légèrement supérieure à celui de l'arbre de décision.

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
y_pred_RFC= train_and_evaluate_model(rf_model, "Random Forest", X_train, y_train, X_val, y_val)
# y_pred_RFC = rf_model.predict(X_val)
# evaluate_model(y_val, y_pred_RFC, "Random Forest")


Random Forest
accuracy : 0.7613
precision : [0.8226 0.5973]
recall : [0.8453 0.5571]
f1 score : [0.8338 0.5765]


In [ ]:
feat_imp = pd.DataFrame({'importance': rf_model.feature_importances_}, index=rf_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.820700
Stat_resp_rep,0.053454
Bor_resp_rep,0.020499
ratioStd,0.020251
Bor_inc_rep,0.016090
inner,0.008675
H1117,0.006616
H26,0.006307
PropCat_Outdoor,0.004002
PropCat_Dwelling,0.003450


### Recherche des hyper paramètres

#### Recherche #1

Nous avons fait une première recherche d'hyper-paramètres en nous basant sur un jeu d'entraînement d'environ 140 000 observations.

In [ ]:
rf_param_distributions = {
    'n_estimators': [100, 200, 300, 500], # nombre d'arbres maximal dans la forêt
    'max_depth': [None, 10, 20, 30, 50], # profondeur maximale de chaque arbre
    'min_samples_split': [2, 5, 10], # nombre minimum d'observations requis pour diviser un nœud
    'min_samples_leaf': [1, 2, 4], # nombre minimum d'observations dans une feuille
    'bootstrap': [True, False],
}


In [ ]:
rf_model_best = RandomForestClassifier(random_state=42)

y_pred_BestRF, best_params_RF= find_best_params_and_predict(rf_model_best, rf_param_distributions, X_train_r, y_train_r, X_val_r, y_val_r, "Optimized Random Forest Regressor", 50)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().




Optimized Random Forest Regressor - Best Parameters: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 20, 'bootstrap': False}
Optimized Random Forest Regressor - Best Cross-Validation Score: 0.8040

Optimized Random Forest Regressor
accuracy : 0.8021
precision : [0.8262 0.7226]
recall : [0.9073 0.5585]
f1 score : [0.8649 0.6301]


L'accuracy sur le jeu de données "complet" est de 80,7%. Les 9 premières variables explicatives avec le plus d'impact sont identiques au modèle de départ.

In [ ]:
best_rf_model = RandomForestClassifier(random_state=42, n_estimators = 200, min_samples_split = 10, min_samples_leaf = 4, max_depth = 20, bootstrap = False)
y_pred_BestRF= train_and_evaluate_model(best_rf_model, "Optimized Random Forest Regressor", X_train, y_train, X_val, y_val)



Optimized Random Forest Regressor
accuracy : 0.8068
precision : [0.8308 0.7222]
recall : [0.9132 0.5484]
f1 score : [0.8701 0.6234]


In [ ]:
feat_imp = pd.DataFrame({'importance': best_rf_model.feature_importances_}, index=best_rf_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.660658
Stat_resp_rep,0.125078
Bor_resp_rep,0.050392
ratioStd,0.036644
Bor_inc_rep,0.028789
inner,0.018478
H1117,0.008270
H26,0.007931
PropCat_Outdoor,0.007681
Borough_E09000022,0.004357


#### Recherche #2

Nous avons fait une seconde recherche en nous basant sur un jeu de données avec environ 215 000 observations pour l'entrainement et 46 000 pour la validation. Nous avons alors utilisé la méthode `find_best_params_and_predict2` (5 datasets pour la cross validation et non 3) et augmenté le nombre d'itérations (de 50 à 100) afin d'explorer plus de valeurs possibles dans le set des hyper-paramètres. Concernant les hyper_paramètres, nous avons ajouté des valeurs possibles à `min_samples_leaf` en indiquant non plus seulement un nombre d'observations mais un pourcentage par rapport au total des observations.

In [ ]:
rf_param_distributions = {
    'n_estimators': [100, 200, 300, 500], # nombre d'arbres maximal dans la forêt
    'max_depth': [None, 10, 20, 30, 50],  # profondeur maximale de chaque arbre
    'min_samples_split': [2, 5, 10], # nombre minimum d'observations nécessaire pour diviser un nœud
    'min_samples_leaf': [0.0001, 0.001, 0.025, 0.05, 1, 2, 4], # nombre minimum d'observations dans une feuille
    'bootstrap': [True, False],
}

model = RandomForestClassifier(random_state=42, n_estimators = 200, min_samples_split = 10, min_samples_leaf = 4, max_depth = 20, bootstrap = False)
y_pred_BestRF, best_params_RF = find_best_params_and_predict2(model, rf_param_distributions, X_train_r2, y_train_r2, X_val_r2, y_val_r2, "RF Random Search", 100)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().




RF Random Search - Best Parameters: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 0.0001, 'max_depth': 30, 'bootstrap': False}
RF Random Search - Best Cross-Validation Score: 0.8057

RF Random Search
accuracy : 0.8049
precision : [0.827  0.7298]
recall : [0.9125 0.5531]
f1 score : [0.8676 0.6293]


Avec ce jeu d'hyper paramètres, l'accuracy est de 80,5% - soit légèrement inférieur à celle obtenue avec le premier set d'hyper paramètres optimisé. De plus, il y a une inconsitence dans ce set puis que le nombre minimal d'observations  pour diviser un noeud est de 2 (min_samples_split) alors que le nombre minimal d'observations dans une feuille est de 0,01% soit 21 pour le jeu de données réduit et 72 pour le complet. Nous n'avons donc pas conservé ce modèle dans la suite de l'analyse.

In [ ]:
best_rf_model = RandomForestClassifier(random_state=42, n_estimators = 500, min_samples_split = 2,
                                       min_samples_leaf = 0.0001, max_depth = 30, bootstrap = False)
y_pred_BestRF= train_and_evaluate_model(best_rf_model, "Optimized Random Forest Regressor #2", X_train, y_train, X_val, y_val)


Optimized Random Forest Regressor #2
accuracy : 0.8054
precision : [0.8271 0.7259]
recall : [0.9169 0.5343]
f1 score : [0.8697 0.6156]


In [ ]:
feat_imp = pd.DataFrame({'importance': best_rf_model.feature_importances_}, index=best_rf_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.614500
Stat_resp_rep,0.146831
Bor_resp_rep,0.060808
ratioStd,0.043074
Bor_inc_rep,0.032884
inner,0.024205
H1117,0.007773
PropCat_Outdoor,0.007670
H26,0.006799
Borough_E09000022,0.005186


## <font color='blue'>4.f) Gradient Boosting </font>

### Première estimation

Pour le première estimation avec le modèle Gradient Boosting, nous avons fixé le nombre d'arbres maximal à 100, la profondeur maximale des arbres à 5 et le taux d'apprentissage à 0.1. L'accuracy de ce modèle est de 80,5%.

In [ ]:
gbm_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
y_pred_GBC= train_and_evaluate_model(gbm_model, "Gradient Boosting", X_train, y_train, X_val, y_val)


Gradient Boosting
accuracy : 0.8047
precision : [0.8272 0.723 ]
recall : [0.9156 0.5354]
f1 score : [0.8691 0.6152]


In [ ]:
feat_imp = pd.DataFrame({'importance': gbm_model.feature_importances_}, index=gbm_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.917968
Stat_resp_rep,0.010866
H26,0.009321
ratioStd,0.007494
Bor_inc_rep,0.005842
Borough_E09000033,0.005146
PropCat_Outdoor,0.005023
H1117,0.004587
PropCat_Other Residential,0.004267
Borough_E09000022,0.003727


### Recherche des hyper paramètres

Nous avons suivi la même procédure de recherches d'hyper paramètres que précédemment (et ce aussi pour les modèles suivant) :
- 1ère recherche d'hyper paramètres sur un jeu de données réduit avec 140 000 observations (pour le jeu d'entraînement)
- entraînement du modèle avec les hyper paramètres optimisés sur le jeu de données complet
- 2nd recherche d'hyper paramètres sur un jeu de données réduit avec 215 000 observations après modification du domaine de recherche en fonction des premiers résultats
- entraînement du modèle avec le second set d'hyper paramètres optimisés sur le jeu de données complet

#### Recherche #1

In [ ]:
gbm_param_dist = {
    'learning_rate': np.logspace(-3, 0, 10), # % contribution de chaque apprenant faible
    'n_estimators': np.arange(100, 500, 50), # nd d'arbres
    'subsample': np.linspace(0.5, 1, 5),   # % de lignes utilisées pour la construction de chaque arbre
    'max_depth': np.arange(3, 15, 2), # profondeur maximale des arbres
    'min_samples_split' : np.linspace(0.001, 0.05, 5), # % d'observations utilisé pour créer un noeud dans l'arbre
    'max_features' : [25,38, None] # nb de variables explicatives à considérer pour créer un noeud
}

model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
y_pred_XGB_opt, best_params_XGB = find_best_params_and_predict(model, gbm_param_dist, X_train_r, y_train_r, X_val_r, y_val_r, model_name="Gradient Boosting", n_iter=50)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().




Gradient Boosting - Best Parameters: {'subsample': 0.875, 'n_estimators': 300, 'min_samples_split': 0.001, 'max_features': None, 'max_depth': 7, 'learning_rate': 0.021544346900318832}
Gradient Boosting - Best Cross-Validation Score: 0.8047

Gradient Boosting
accuracy : 0.804
precision : [0.8257 0.731 ]
recall : [0.9118 0.5548]
f1 score : [0.8666 0.6308]


L'accuracy est très légèrement amélioré. Le temps d'exécution augmente car le nombre maximal d'arbres est triplé.

In [ ]:
gbm_model = GradientBoostingClassifier(n_estimators=300, learning_rate=0.0215443469,
                                       subsample=0.875, min_samples_split=0.001, max_depth=7,
                                       max_features=None, random_state=42)
y_pred_GBC= train_and_evaluate_model(gbm_model, "Gradient Boosting", X_train, y_train, X_val, y_val)


Gradient Boosting
accuracy : 0.8053
precision : [0.828  0.7232]
recall : [0.9152 0.5383]
f1 score : [0.8694 0.6172]


In [ ]:
feat_imp = pd.DataFrame({'importance': gbm_model.feature_importances_}, index=gbm_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.906365
Stat_resp_rep,0.010188
H26,0.008940
ratioStd,0.008385
Bor_inc_rep,0.007062
Borough_E09000033,0.005521
H1117,0.005401
PropCat_Outdoor,0.004965
PropCat_Other Residential,0.004647
Borough_E09000022,0.003467


#### Recherche #2

In [ ]:
gbm_param_dist = {
    'learning_rate': np.logspace(-2, 0, 10), # % contribution de chaque apprenant faible
    'n_estimators': np.arange(300, 550, 50), # nd d'arbres
    'subsample': np.linspace(0.75, 1, 5),   # % de lignes utilisées pour la construction de chaque arbre
    'max_depth': np.arange(5, 11, 1), # profondeur maximale des arbres
    'min_samples_split' : [0.00025, 0.0005, 0.001, 0.025, 0.005], # % d'observations utilisé pour créer un noeud dans l'arbre
    'max_features' : [25,38, None] # nb de variables explicatives à considérer pour créer un noeud
}

model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
y_pred_XGB_opt, best_params_XGB = find_best_params_and_predict2(model, gbm_param_dist, X_train_r2, y_train_r2, X_val_r2, y_val_r2, model_name="Gradient Boosting", n_iter=100)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().




Gradient Boosting - Best Parameters: {'subsample': 0.875, 'n_estimators': 400, 'min_samples_split': 0.0005, 'max_features': 25, 'max_depth': 9, 'learning_rate': 0.046415888336127774}
Gradient Boosting - Best Cross-Validation Score: 0.8063

Gradient Boosting
accuracy : 0.8061
precision : [0.8294 0.7282]
recall : [0.9104 0.5617]
f1 score : [0.868  0.6342]


L'accuracy est de nouveau amélioré (80,7%). Le temps d'exécution est à nouveau augmenté avec un nombre maximal d'arbres encore augmenté.

In [ ]:
gbm_model = GradientBoostingClassifier(n_estimators=400, learning_rate=0.046415888336,
                                       subsample=0.875, min_samples_split=0.0005, max_depth=9,
                                       max_features=25, random_state=42)
y_pred_GBC= train_and_evaluate_model(gbm_model, "Optimized Gradient Boosting #2", X_train, y_train, X_val, y_val)


Optimized Gradient Boosting #2
accuracy : 0.8071
precision : [0.8308 0.7234]
recall : [0.9137 0.5482]
f1 score : [0.8703 0.6237]


In [ ]:
feat_imp = pd.DataFrame({'importance': gbm_model.feature_importances_}, index=gbm_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.711474
Stat_resp_rep,0.102822
Bor_resp_rep,0.045181
ratioStd,0.029505
Bor_inc_rep,0.017260
inner,0.012245
H26,0.009406
Borough_E09000022,0.006492
H1117,0.006438
PropCat_Outdoor,0.005910


## <font color='blue'>4.g) XG Boost </font>

### Première estimation

Tout comme pour le modèle Gradient Boost, nous avons fixé le nombre d'arbres maximal à 100, la profondeur maximale des arbres à 5 et le taux d'apprentissage à 0.1. L'accuracy de ce modèle est de 80,4%.

In [ ]:
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, use_label_encoder=False)
y_pred_XGBC= train_and_evaluate_model(xgb_model, "XGBoost", X_train, y_train, X_val, y_val)

C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning:

[01:29:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





XGBoost
accuracy : 0.804
precision : [0.8268 0.721 ]
recall : [0.9148 0.5347]
f1 score : [0.8686 0.614 ]


In [ ]:
feat_imp = pd.DataFrame({'importance': xgb_model.feature_importances_}, index=xgb_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)


,importance
distStd,0.350375
Borough_E09000022,0.032974
Borough_E09000009,0.032331
PropCat_Other Residential,0.031691
PropCat_Outdoor,0.030483
Borough_E09000033,0.029465
H26,0.028037
Borough_E09000005,0.025748
Stat_resp_rep,0.025619
Borough_E09000010,0.024395


### Recherche des hyper paramètres

#### Recherche #1

Après cette première recherche, l'accuracy est de 80,7%. Malgré l'augmentation du nombre maximal d'arbre, l'exécution de ce modèle est rapide.

In [ ]:
xgb_param_dist = {
    'n_estimators': np.arange(100, 500, 50),
    'max_depth': np.arange(3, 15, 2),
    'learning_rate': np.logspace(-3, 0, 10),
    'subsample': np.linspace(0.5, 1, 5),
    'colsample_bytree': np.linspace(0.5, 1, 5),
    'gamma': np.linspace(0, 1, 5),
    'reg_alpha': np.logspace(-3, 1, 5),
    'reg_lambda': np.logspace(-3, 1, 5)
}

model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, use_label_encoder=False)
y_pred_XGB_opt, best_params_XGB = find_best_params_and_predict(model, xgb_param_dist, X_train_r, y_train_r, X_val_r, y_val_r, model_name="XGB", n_iter=50)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning:

[19:50:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





XGB - Best Parameters: {'subsample': 0.875, 'reg_lambda': 0.1, 'reg_alpha': 0.01, 'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.046415888336127774, 'gamma': 1.0, 'colsample_bytree': 0.5}
XGB - Best Cross-Validation Score: 0.8047

XGB
accuracy : 0.804
precision : [0.8271 0.7276]
recall : [0.9093 0.5603]
f1 score : [0.8663 0.6331]


In [ ]:
xgb_model = XGBClassifier(subsample=0.875, reg_lambda=0.1, reg_alpha=0.01, random_state=42, use_label_encoder=False, n_estimators=300,
                     max_depth=9, learning_rate=0.046415888, gamma=1, colsample_bytree=0.5)
y_pred_XGBC= train_and_evaluate_model(xgb_model, "XGBoost", X_train, y_train, X_val, y_val)

C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning:

[19:53:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





XGBoost
accuracy : 0.8066
precision : [0.8304 0.7228]
recall : [0.9137 0.5466]
f1 score : [0.87   0.6225]


In [ ]:
feat_imp = pd.DataFrame({'importance': xgb_model.feature_importances_}, index=xgb_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
Stat_resp_rep,0.216213
inner,0.127972
distStd,0.100227
Bor_resp_rep,0.054842
Borough_E09000033,0.027397
ratioStd,0.020597
Borough_E09000017,0.018919
H26,0.018566
PropCat_Outdoor,0.017681
Borough_E09000020,0.017442


#### Recherche #2

L'accuracy est légèrement augmentée avec ce jeu d'hyper paramètres. Les 6 paramètres les plus importants sont identiques à ceux du modèle précédent.

In [ ]:
xgb_param_dist = {
    'n_estimators': np.arange(300, 550, 50),
    'max_depth': np.arange(5, 11, 1),
    'learning_rate': np.logspace(-3, 0, 10),
    'subsample': np.linspace(0.75, 1, 5),
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 1],
    'gamma': [0, 0.5, 1, 2, 5], # paramètre de régularisation qui contrôle la complexité de l'arbre de décision / son élagage
    'reg_alpha': [0, 0.001, 0.005, 0.01, 0.05, 0.1],
    'reg_lambda': [0, 0.01, 0.05, 0.1, 0.25, 0.5],
    'scale_pos_weight': [0, 1, 2, 2.5, 5, 10] # ajuster le déséquilibre entre les classes

}

model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, use_label_encoder=False)
y_pred_XGB_opt, best_params_XGB = find_best_params_and_predict2(model, xgb_param_dist, X_train_r2, y_train_r2, X_val_r2, y_val_r2, model_name="XGB", n_iter=100)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning:

[19:36:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





XGB - Best Parameters: {'subsample': 0.8125, 'scale_pos_weight': 1, 'reg_lambda': 0.01, 'reg_alpha': 0.05, 'n_estimators': 350, 'max_depth': 10, 'learning_rate': 0.046415888336127774, 'gamma': 2, 'colsample_bytree': 0.6}
XGB - Best Cross-Validation Score: 0.8065

XGB
accuracy : 0.8054
precision : [0.8284 0.7283]
recall : [0.911  0.5583]
f1 score : [0.8678 0.6321]


In [ ]:
xgb_model = XGBClassifier(subsample=0.8125, reg_lambda=0.01, reg_alpha=0.05, random_state=42,
                          use_label_encoder=False, n_estimators=350,
                         max_depth=10, learning_rate=0.046415888, gamma=2, colsample_bytree=0.6,
                         scale_pos_weight=1)
y_pred_XGBC= train_and_evaluate_model(xgb_model, "Optimized XGBoost #2", X_train, y_train, X_val, y_val)

C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning:

[00:03:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





Optimized XGBoost #2
accuracy : 0.807
precision : [0.8305 0.7237]
recall : [0.914 0.547]
f1 score : [0.8703 0.623 ]


In [ ]:
feat_imp = pd.DataFrame({'importance': xgb_model.feature_importances_}, index=xgb_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
Stat_resp_rep,0.231240
inner,0.117555
distStd,0.105834
Bor_resp_rep,0.043652
Borough_E09000033,0.026365
ratioStd,0.020114
H26,0.018462
Borough_E09000017,0.016687
Borough_E09000020,0.016474
PropCat_Outdoor,0.015655


## <font color='blue'>4.h) Light Gradient Boosting </font>

### Première estimation

Tout comme pour le modèle Gradient Boost, nous avons fixé le nombre d'arbres maximal à 100, la profondeur maximale des arbres à 5 et le taux d'apprentissage à 0.1. L'accuracy de ce modèle est de 80,5%.

In [ ]:
lgbm_model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)
y_pred_LGBMC= train_and_evaluate_model(lgbm_model, "LightGBM", X_train, y_train, X_val, y_val)

C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py:150: UserWarning:

Could not find the number of physical cores for the following reason:
[WinError 2] Le fichier spécifié est introuvable
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.

  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py", line 227, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\subprocess.py", line 1440, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(execu

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 213512, number of negative: 512887
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.503242 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 383
[LightGBM] [Info] Number of data points in the train set: 726399, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.293932 -> initscore=-0.876363
[LightGBM] [Info] Start training from score -0.876363

LightGBM
accuracy : 0.8048
precision : [0.8282 0.7207]
recall : [0.9139 0.5397]
f1 score : [0.869  0.6172]


In [ ]:
feat_imp = pd.DataFrame({'importance': lgbm_model.feature_importances_}, index=lgbm_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,980
Bor_inc_rep,173
H26,152
ratioStd,144
Stat_resp_rep,140
PropCat_Dwelling,107
PropCat_Non_Residential,105
Borough_E09000033,93
H1117,90
PropCat_Other_Residential,89


### Recherche des hyper paramètres

#### Recherche #1

Après cette première recherche, l'accuracy est de 80,6% sur le jeu de données complet.

In [ ]:
lgbm_param_dist = {
    'n_estimators': np.arange(100, 500, 50),
    'max_depth': np.arange(3, 15, 2),
    'learning_rate': np.logspace(-3, 0, 10),
    'subsample': np.linspace(0.5, 1, 5),
    'colsample_bytree': np.linspace(0.5, 1, 5),
    'reg_alpha': np.logspace(-3, 1, 5),
    'reg_lambda': np.logspace(-3, 1, 5),
    'num_leaves': np.arange(20, 150, 10)
}

model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)
y_pred_LGBM_opt, best_params_LGBM = find_best_params_and_predict(model, lgbm_param_dist, X_train_r, y_train_r, X_val_r, y_val_r, model_name="LGBM", n_iter=50)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py:150: UserWarning:

Could not find the number of physical cores for the following reason:
[WinError 2] Le fichier spécifié est introuvable
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.

  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py"

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 42347, number of negative: 97642
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006106 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 384
[LightGBM] [Info] Number of data points in the train set: 139989, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302502 -> initscore=-0.835410
[LightGBM] [Info] Start training from score -0.835410
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f


XGB
accuracy : 0.8032
precision : [0.8267 0.7258]
recall : [0.9087 0.5592]
f1 score : [0.8657 0.6317]


In [ ]:
lgbm_model = LGBMClassifier(random_state=42, subsample=0.75, reg_lambda=0.01, reg_alpha=0.1, num_leaves=90, n_estimators=450,
              max_depth=7, learning_rate=0.046415888, colsample_bytree=0.875)
y_pred_LGBMC= train_and_evaluate_model(lgbm_model, "LightGBM", X_train, y_train, X_val, y_val)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 213512, number of negative: 512887
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028881 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 383
[LightGBM] [Info] Number of data points in the train set: 726399, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.293932 -> initscore=-0.876363
[LightGBM] [Info] Start training from score -0.876363
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
feat_imp = pd.DataFrame({'importance': lgbm_model.feature_importances_}, index=lgbm_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,14336
ratioStd,2101
Bor_inc_rep,1871
H1117,1789
Stat_resp_rep,1459
PropCat_Non_Residential,1418
Bor_resp_rep,1305
PropCat_Dwelling,1221
H26,1145
PropCat_Other_Residential,846


#### Recherche #2

Après cette première recherche, l'accuracy est de 80,7% sur le jeu de données complet.

In [ ]:
lgbm_param_dist = {
    'n_estimators': np.arange(100, 500, 50),
    'max_depth': np.arange(3, 15, 2),
    'learning_rate': np.logspace(-3, 0, 10),
    'subsample': np.linspace(0.5, 1, 5),
    'colsample_bytree': [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1],
    'reg_alpha': [0, 0.01, 0.05, 0.1, 0.25, 0.5],
    'reg_lambda': [0, 0.001, 0.005, 0.01, 0.05, 0.1],
    'num_leaves': [75,  80,  90, 100, 110, 120, 130],
    'scale_pos_weight': [0, 1, 2, 2.5, 5, 10] # ajuster le déséquilibre entre les classes
}

model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)
y_pred_LGBM_opt, best_params_LGBM = find_best_params_and_predict2(model, lgbm_param_dist, X_train_r2, y_train_r2, X_val_r2, y_val_r2, model_name="LGBM", n_iter=100)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning:


80 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 1284, in fit
    super().fit(
  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 955, in fit
    self._Booster = train(
  File "C:\Users\AD

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 64315, number of negative: 150546
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022420 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 384
[LightGBM] [Info] Number of data points in the train set: 214861, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.299333 -> initscore=-0.850476
[LightGBM] [Info] Start training from score -0.850476

XGB - Best Parameters: {'subsample': 0.875, 'scale_pos_weight': 1, 'reg_lambda': 0, 'reg_alpha': 0.5, 'num_leaves': 80, 'n_estimators': 250, 'max_depth': 13, 'learning_rate': 0.1, 'colsample_bytree': 0.95}
XGB - Best Cross-Validation Score: 0.8060

XGB
accuracy : 0.8054
precision : [0.8291 0.7265]
recall : [0.9098 0.5611]
f1 score : [0.8676 0.6332]


In [ ]:
lgbm_model = LGBMClassifier(random_state=42, subsample=0.875, reg_lambda=0, reg_alpha=0.5,
                            num_leaves=80, n_estimators=250, scale_pos_weight=1,
                            max_depth=13, learning_rate=0.1, colsample_bytree=0.95)
y_pred_LGBMC= train_and_evaluate_model(lgbm_model, "Optimized LightGBM #2", X_train, y_train, X_val, y_val)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 213512, number of negative: 512887
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023165 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 383
[LightGBM] [Info] Number of data points in the train set: 726399, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.293932 -> initscore=-0.876363
[LightGBM] [Info] Start training from score -0.876363

Optimized LightGBM #2
accuracy : 0.8072
precision : [0.8307 0.7239]
recall : [0.914  0.5476]
f1 score : [0.8704 0.6235]


In [ ]:
feat_imp = pd.DataFrame({'importance': lgbm_model.feature_importances_}, index=lgbm_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,10428
ratioStd,1163
Bor_inc_rep,727
H1117,594
PropCat_Non_Residential,562
Stat_resp_rep,526
PropCat_Dwelling,524
Bor_resp_rep,454
H26,438
PropCat_Outdoor,405


# <font color='red'> 6) Conclusions
 </font>


Le tableau ci-dessous résume les résultats de modélisation de ce notebook. Les modèles sont classés par accuracy croissante. Les résultats des algorithmes de forêt aléatoire, Gradient Boosting, XG Boost et LGBM sont proches.
</br></br>

<table>
<tr>
<th>Modèle</th>
<th>Accuracy</th>
<th>Precision_0</th>
<th>Precision_1</th>
<th>Recall_0</th>
<th>Recall_1</th>  
<th>F1score_0</th>    
<th>F1score_1</th>           
</tr>

<tr>
<td>Arbre de décision</td>
<td> 0.7586</td>
<td> 0.8159</td>
<td> 0.5964</td>
<td> 0.8514</td>
<td> 0.5334</td>
<td>0.8333 </td>
<td> 0.5631</td>  
</tr>

<tr>
<td>Régression logistique</td>
<td>0.7657 </td>
<td>0.7857</td>
<td>0.6684</td>
<td> 0.9202</td>
<td> 0.3905</td>
<td>0.868</td>
<td> 061</td>
</tr>    


<tr>
<td>K-plus proches voisins</td>
<td>0.8028 </td>
<td> 0.8251</td>
<td> 0.7208</td>
<td> 0.9157</td>
<td> 0.5287</td>
<td> 0.868</td>
<td> 0.61</td>  
</tr>  

<tr>
<td> Gradient Boosting #1</td>
<td>0.8053 </td>
<td>0.828 </td>
<td> 0.7232</td>
<td> 0.9152</td>
<td> 0.5383</td>
<td> 0.8694</td>
<td> 0.6172</td>  
</tr>

<tr>
<td>Forêt aléatoire #2</td>
<td> 0.8054</td>
<td> 0.8271</td>
<td> 0.7259</td>
<td> 0.9169</td>
<td> 0.5343</td>
<td> 0.8697</td>
<td> 0.6156</td>  
</tr>

<tr>
<td>Light Gradient Boosting #1</td>
<td>0.8061 </td>
<td> 0.83</td>
<td> 0.7217</td>
<td> 0.9134</td>
<td> 0.5454</td>
<td> 0.8697</td>
<td> 0.6213</td>  
</tr>

<tr>
<td>XG Boost #1</td>
<td> 0.8066</td>
<td> 0.8304</td>
<td> 0.7228</td>
<td> 0.9137</td>
<td> 0.5466</td>
<td> 0.87</td>
<td> 0.6225</td>  
</tr>

<tr>
<td>Forêt aléatoire #1</td>
<td> 0.8068</td>
<td> 0.8308</td>
<td> 0.7222</td>
<td> 0.9132</td>
<td> 0.5484</td>
<td> 0.8701</td>
<td> 0.6234</td>  
</tr>

<tr>
<td>XG Boost #2</td>
<td> 0.8070</td>
<td> 0.8305</td>
<td> 0.7237</td>
<td> 0.914</td>
<td> 0.547</td>
<td> 0.8703</td>
<td> 0.623</td>  
</tr>
    


<tr>
<td> Gradient Boosting #2</td>
<td>0.8071 </td>
<td>0.8308 </td>
<td> 0.7234</td>
<td> 0.9137</td>
<td> 0.5482</td>
<td> 0.8703</td>
<td> 0.6237</td>  
</tr>
    



    

    
<tr>
<td>Light Gradient Boostin #2</td>
<td>0.8072 </td>
<td> 0.8307</td>
<td> 0.7239</td>
<td> 0.914</td>
<td> 0.5476</td>
<td> 0.8704</td>
<td> 0.6235</td>  
</tr>    


Dans le notebook suivant nous appliquons ces 4 modèles aux données "finales". Pour rappel, nous avons divisé le jeu de données en 3 : *train*, *validation* et *test*. Dans les notebooks 6 à 9, nous avons utilisé les deux premiers. Dans le notebook suivant, nous regroupons les jeux *train* et *validation* dans un nouveau jeu d'entraînement et nous utilisons *test* pour la validation du modèle.